In [109]:
from copy import deepcopy
from collections import defaultdict
import random
import numpy as np
import time 
import json

## Reading and initializing functions

In [102]:
def get_sudokus(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    returns a list of sudoku board states as clauses in a dict
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for i, state in enumerate(sudoku_states):
            sudoku = {}
            pointers = {}
            row = 1
            index = 0
            for j, content in enumerate(state):
                if j%sudokusize == 0 and j > 0:
                    row+=1
                try:
                    int(content)
                    literal = int(str(row) + str(j%sudokusize+1) + content)
                    sudoku[index] = [literal]
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    index += 1
                except:
                    # is not convertible to int
                    continue
            sudokus.append((sudoku, pointers))
    return sudokus

def get_constraints_from_dimacs(filename, prevConstraints={}, prevPointers={}):
    '''
    Convert dimacs file to a dictionary of constraints
    and pointers. Allows merging with previous pointers and constraints.
    '''
    constraints = prevConstraints # dictionary with constraints as: clausenumber->[literals]
    pointers = prevPointers # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    initial_length = len(constraints)
    with open(filename) as f:
        for i, clause in enumerate(f):
            index = i + initial_length
            if clause[0] == 'c' or clause[0] == 'p':
                # skip comments etc.
                continue
            constraints[index]= []
            for literal in clause.split():
                literal = int(literal)
                if literal != 0:
                    constraints[index].append(literal)
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    
    return constraints, pointers

    

## DP helper functions for constraints updates

In [103]:
def assign_literal(constraints, pointers, literal, assignments, random=False):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    if random:
        extract_features(constraints, constraintsUpdated, pointers, pointersUpdated, assignments, literal)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literal):
    '''
    literal: e.g. 112 or 112'
    Updates constraints and pointers by removing all clauses 
    with the given literal, because the clause becomes true.
    Remove counter-literal out of all clauses.
    '''
    ## Find clauses with literal and remove them
    clauses = list(pointers[literal])
    for clause in clauses:
        for lit in constraints[clause]:
            pointers[lit].remove(clause)
        del constraints[clause]
    del pointers[literal]
    
    ## Find clauses with counter-literal and remove its counter-literal
    counterLiteral = -literal
    clauses = list(pointers[counterLiteral])
    del pointers[counterLiteral]
    for clause in clauses:
        constraints[clause].remove(counterLiteral)

    return constraints, pointers

def update_assigments(assignments, literal):
    '''
    Keep track of assigned literals.
    Needed in order to return satisfiable solution of literals.
    '''
    if literal in assignments:
        print("already assigned", literal, "(in this stack), returning...")
        return
    else:
        assignments.append(literal)
        return assignments
    
def check_tautologies(constraints, pointers):
    '''
    Check for clauses with a tautology and remove
    them.
    '''
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

### Extract features to use as regressor datapoints

In [113]:
def extract_features(constraintsOld, constraintsNew, pointersOld, pointersNew, assignments, literal):
    features[literal] = {}
    
    ## Ratio of the polarity of this literal before assignment
    positive = len(pointersOld[literal])
    negative = len(pointersOld[-literal])
    ratio = positive/(positive+negative)
    features[literal]["posneg_ratio"] = ratio
    
    ## Clauses that were resolved with this assignment
    clausesResolved = len(constraintsOld) - len(constraintsNew)
    features[literal]["c_res"] = clausesResolved
    
    ## Clauses that became empty with this assignment
    emptyClauses = 0
    for clause in constraintsNew.keys():
        if not constraintsNew[clause]:
            emptyClauses += 1
    features[literal]["empty_c"] = emptyClauses
    
    ## Average clause length with given literal before assignment
    clauses = pointersOld[literal]
    lengths = [len(constraintsOld[c]) for c in clauses]
    features[literal]["c_len"] = np.mean(lengths) 
    
    ## Label whether this assignment results in succes (to be determined at halt)
    features[literal]["SAT"] = 0
    

## DP cases

In [105]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clause(constraints): 
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = next(iter(constraints[clause]))
            return literal

    return False

def unit_propagate(constraints, pointers, assignments):
    '''
    Automatically assign the literal of all unit clauses
    until there are no unit clauses left.
    '''
    literal = unit_clause(constraints)
    while(not empty_clause(constraints) and literal):
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        literal = unit_clause(constraints)
    return constraints, pointers, assignments

def pure_literal(literals):
    for literal in literals:
        if -literal not in literals:
            return literal
        
    return False

def random_literal(literals): 
    literal = random.choice(literals) 
    return literal

## DP main function

In [106]:
def DP(constraints, pointers, assignments): 
    '''
    DP main recursive function. Always returns satisfiable
    assignment for all constraints if there is one.
    '''
    constraints, pointers, assignments = unit_propagate(constraints, pointers, assignments)
    
    if len(constraints)==0:
        return assignments
   
    if empty_clause(constraints):
        return False
    
    all_literals = list(pointers.keys())
    
    literal = pure_literal(all_literals)
    if literal:
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        DP(constraints, pointers, assignments)
    
    literal = random_literal(all_literals)
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments, random=True)
    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated)
    if assignmentsFinal:
        return assignmentsFinal
    else:
        constraints, pointers, assignments = assign_literal(constraints, pointers, -literal, assignments, random=True)
        return DP(constraints, pointers, assignments)

### Script

In [114]:
    datapoints = []
    
    train_sudokus = [x for x in range(0,1000) if x%2 == 0]
    
    ## Run DP
    for i in train_sudokus:
        ## Get board and rules
        startTime = time.time()
        print("Reading and analyzing...")
        constraints, pointers = get_sudokus('1000 sudokus.txt')[i]
        #     constraints, pointers = get_constraints_from_dimacs('sudoku-example.txt')
        constraints, pointers = get_constraints_from_dimacs('sudoku-rules.txt', constraints, pointers)
        constraints, pointers = check_tautologies(constraints, pointers)
        print("Starting DP algorithm...")
        global features
        features = {}
        assignments = DP(constraints, pointers, [])
        endTime = time.time()
        if assignments:
            print("Solution found")
            print("Time to complete:", endTime-startTime)
            boardPositions = [a for a in assignments if a > 0]
            print(boardPositions)
            for a in assignments:
                if a in features.keys():
                    features[a]["SAT"] = 1
            for f in features.keys():
                datapoints.append(features[f])
        else:
            print("No solution found")
            
    with open('data.json', 'w') as outfile:
        json.dump(datapoints, outfile)

Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.68959951400757
[129, 134, 171, 183, 357, 366, 392, 428, 451, 523, 532, 642, 686, 755, 774, 868, 897, 936, 943, 964, 998, 226, 196, 117, 212, 927, 876, 746, 767, 841, 261, 253, 416, 556, 654, 548, 514, 824, 722, 447, 637, 145, 162, 158, 793, 678, 473, 482, 494, 663, 972, 852, 959, 238, 388, 284, 277, 249, 344, 295, 379, 718, 575, 587, 591, 699, 569, 465, 439, 819, 833, 915, 731, 789, 981, 885, 313, 611, 625, 321, 335]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.56543254852295
[167, 229, 261, 354, 365, 396, 452, 523, 536, 574, 581, 615, 678, 699, 794, 851, 868, 928, 931, 945, 983, 992, 555, 762, 569, 788, 771, 191, 298, 158, 597, 682, 512, 548, 493, 663, 895, 624, 637, 421, 641, 447, 464, 656, 311, 338, 418, 439, 844, 966, 914, 253, 373, 743, 735, 125, 234, 184, 827, 832, 813, 726, 719, 757, 959, 977, 116, 133, 322, 217,

No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.617212533950806
[146, 194, 214, 225, 272, 358, 369, 373, 419, 427, 493, 526, 533, 634, 662, 677, 716, 739, 755, 915, 963, 986, 991, 344, 435, 512, 129, 113, 165, 152, 482, 476, 656, 643, 545, 559, 567, 253, 866, 261, 247, 875, 979, 849, 598, 299, 396, 385, 187, 171, 288, 138, 236, 574, 581, 778, 764, 957, 851, 742, 948, 721, 823, 924, 818, 797, 892, 783, 884, 837, 932, 317, 611, 628, 322, 331, 441, 454, 468, 689, 695]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 23.201345920562744
[156, 225, 279, 287, 332, 365, 442, 488, 527, 534, 628, 635, 654, 672, 691, 731, 767, 816, 864, 919, 922, 946, 971, 512, 715, 613, 411, 823, 724, 647, 357, 117, 937, 838, 752, 262, 241, 851, 561, 953, 968, 845, 749, 877, 169, 121, 381, 133, 236, 329, 426, 439, 666, 463, 455, 689, 343, 258, 144, 293, 214, 318, 548, 559, 474, 497, 376, 394, 984, 995, 882, 899, 185, 192, 17

Solution found
Time to complete: 20.824254751205444
[161, 178, 185, 232, 268, 283, 291, 313, 345, 374, 416, 424, 431, 519, 565, 688, 749, 766, 915, 939, 954, 987, 992, 225, 635, 321, 338, 129, 136, 528, 664, 963, 926, 971, 948, 458, 581, 594, 784, 834, 197, 114, 217, 244, 612, 382, 489, 659, 641, 546, 256, 279, 396, 357, 369, 886, 899, 798, 711, 818, 851, 755, 722, 773, 875, 737, 823, 627, 533, 552, 467, 443, 577, 142, 153, 847, 862, 472, 495, 676, 693]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.190358638763428
[131, 188, 264, 275, 335, 357, 432, 447, 453, 572, 613, 625, 638, 651, 712, 733, 755, 766, 819, 891, 936, 959, 977, 915, 727, 921, 834, 828, 741, 361, 554, 944, 968, 852, 843, 867, 163, 145, 258, 318, 548, 156, 646, 662, 471, 511, 537, 239, 242, 349, 373, 223, 876, 885, 778, 281, 687, 498, 595, 583, 486, 465, 569, 414, 429, 526, 117, 216, 297, 982, 993, 384, 179, 396, 192, 124, 322, 674, 699, 789, 794]
Reading and a

No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.239155292510986
[118, 163, 185, 197, 217, 235, 266, 273, 332, 364, 448, 469, 539, 554, 572, 591, 626, 633, 783, 834, 887, 912, 959, 515, 528, 422, 437, 567, 677, 282, 586, 493, 543, 456, 411, 614, 853, 923, 727, 825, 947, 744, 313, 136, 846, 819, 716, 357, 345, 258, 484, 475, 396, 976, 994, 795, 892, 779, 965, 655, 174, 299, 224, 241, 152, 149, 121, 329, 642, 661, 751, 868, 762, 738, 931, 871, 988, 378, 381, 689, 698]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.60735034942627
[161, 211, 238, 254, 275, 326, 359, 374, 469, 539, 547, 591, 614, 641, 672, 683, 825, 837, 844, 866, 918, 973, 992, 699, 627, 564, 381, 898, 889, 871, 179, 229, 719, 949, 188, 282, 452, 853, 812, 522, 428, 431, 578, 443, 513, 416, 635, 477, 586, 555, 668, 656, 246, 196, 157, 263, 115, 317, 142, 332, 297, 393, 762, 951, 758, 967, 745, 721, 924, 733, 936, 985, 123, 134, 348

Solution found
Time to complete: 20.733002185821533
[134, 148, 167, 193, 211, 286, 329, 378, 418, 436, 492, 551, 583, 625, 649, 691, 766, 775, 784, 816, 855, 914, 961, 421, 524, 935, 126, 341, 546, 356, 676, 474, 489, 595, 688, 577, 568, 996, 731, 152, 115, 181, 179, 385, 272, 662, 654, 871, 973, 959, 758, 898, 799, 839, 928, 519, 532, 238, 312, 457, 253, 364, 245, 269, 227, 333, 294, 397, 717, 613, 637, 844, 863, 742, 947, 723, 822, 887, 982, 443, 465]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.4451265335083
[124, 156, 171, 187, 245, 252, 311, 398, 438, 495, 548, 563, 572, 616, 622, 637, 679, 721, 796, 834, 855, 862, 947, 261, 531, 442, 846, 641, 168, 951, 654, 665, 466, 758, 353, 149, 367, 344, 743, 481, 891, 586, 597, 474, 514, 525, 457, 559, 693, 688, 192, 332, 889, 994, 764, 969, 284, 273, 299, 376, 385, 236, 329, 926, 978, 423, 419, 739, 877, 983, 782, 775, 912, 717, 935, 813, 828, 115, 133, 218, 227]
Reading and ana

...

Solution found
Time to complete: 21.426219701766968
[151, 182, 198, 264, 312, 333, 349, 365, 387, 468, 473, 497, 539, 556, 613, 679, 688, 734, 769, 796, 829, 832, 867, 881, 954, 972, 459, 358, 777, 878, 894, 119, 391, 571, 486, 584, 414, 644, 293, 289, 999, 742, 948, 961, 846, 853, 755, 783, 985, 923, 815, 238, 441, 662, 563, 435, 422, 592, 695, 545, 657, 631, 626, 324, 174, 376, 275, 125, 252, 143, 166, 247, 137, 216, 221, 711, 728, 917, 936, 518, 527]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 22.27664041519165
[198, 218, 242, 275, 325, 349, 397, 438, 455, 464, 479, 511, 588, 624, 632, 648, 693, 737, 744, 814, 823, 851, 886, 895, 916, 952, 872, 839, 847, 868, 562, 358, 685, 535, 413, 312, 182, 715, 722, 931, 928, 778, 492, 791, 543, 945, 141, 165, 221, 371, 446, 661, 481, 427, 526, 619, 594, 577, 676, 559, 657, 117, 129, 267, 296, 289, 987, 384, 173, 974, 783, 999, 756, 769, 963, 253, 366, 154, 136, 234, 333]
Reading and an

Solution found
Time to complete: 31.792374849319458
[159, 164, 176, 232, 248, 256, 299, 363, 384, 398, 439, 491, 531, 543, 567, 627, 634, 675, 718, 774, 782, 829, 833, 844, 858, 957, 319, 138, 753, 454, 594, 413, 224, 123, 914, 993, 273, 683, 668, 649, 651, 769, 795, 897, 871, 886, 737, 815, 922, 721, 936, 746, 862, 335, 326, 352, 192, 516, 612, 696, 442, 555, 466, 572, 589, 425, 528, 377, 217, 111, 185, 281, 147, 265, 341, 945, 961, 478, 487, 979, 988]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.765987873077393
[168, 243, 271, 284, 298, 324, 386, 392, 437, 451, 495, 535, 556, 567, 594, 628, 665, 716, 731, 782, 832, 913, 934, 946, 979, 981, 818, 338, 373, 266, 239, 962, 691, 896, 793, 997, 885, 874, 778, 958, 925, 175, 677, 476, 572, 636, 126, 133, 521, 519, 423, 653, 583, 689, 488, 548, 222, 152, 144, 111, 642, 449, 464, 412, 614, 863, 841, 361, 359, 754, 769, 745, 857, 727, 829, 347, 255, 217, 315, 187, 199]
Reading and a

No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.059199810028076
[128, 136, 169, 211, 235, 258, 272, 334, 341, 353, 377, 526, 564, 583, 598, 643, 659, 662, 695, 755, 792, 886, 921, 949, 952, 978, 994, 142, 365, 456, 676, 482, 396, 388, 154, 284, 299, 117, 223, 474, 789, 579, 829, 322, 319, 425, 512, 439, 413, 545, 832, 497, 681, 461, 557, 448, 531, 851, 771, 185, 191, 173, 875, 893, 987, 915, 716, 966, 763, 868, 933, 738, 727, 814, 744, 847, 618, 624, 637, 246, 267]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.694384336471558
[163, 182, 244, 296, 327, 334, 345, 366, 393, 425, 438, 483, 511, 548, 577, 622, 684, 747, 791, 815, 846, 854, 872, 928, 931, 965, 976, 861, 773, 785, 174, 994, 586, 442, 499, 592, 471, 555, 732, 952, 943, 653, 641, 456, 213, 312, 118, 149, 262, 759, 768, 197, 275, 135, 239, 126, 221, 151, 358, 257, 288, 381, 379, 724, 716, 414, 467, 564, 669, 617, 636, 919, 837, 823, 89

No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 20.774213552474976
[157, 189, 198, 246, 285, 336, 371, 394, 422, 491, 513, 534, 549, 566, 582, 631, 652, 667, 688, 799, 811, 825, 843, 919, 924, 965, 978, 176, 551, 528, 956, 775, 577, 595, 227, 121, 261, 387, 644, 445, 615, 135, 163, 323, 239, 726, 416, 629, 437, 696, 479, 484, 673, 717, 947, 741, 981, 897, 886, 142, 254, 114, 218, 312, 348, 355, 369, 453, 468, 758, 838, 859, 272, 293, 874, 992, 783, 732, 933, 764, 862]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.272947549819946
[141, 157, 174, 183, 233, 269, 291, 342, 395, 517, 552, 563, 594, 623, 636, 655, 664, 677, 718, 735, 743, 832, 845, 934, 959, 976, 998, 985, 913, 337, 947, 962, 921, 311, 244, 324, 414, 353, 165, 215, 467, 451, 531, 287, 681, 648, 473, 575, 425, 612, 699, 893, 797, 827, 761, 871, 379, 278, 192, 386, 222, 256, 368, 754, 884, 726, 819, 116, 858, 866, 772, 789, 482, 496, 58

No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.546501874923706
[225, 234, 259, 286, 297, 331, 352, 384, 398, 446, 517, 523, 594, 615, 628, 683, 726, 749, 767, 824, 848, 856, 887, 899, 938, 954, 993, 414, 927, 919, 421, 122, 272, 329, 137, 347, 774, 976, 536, 696, 788, 542, 632, 439, 669, 644, 164, 366, 116, 313, 218, 158, 145, 173, 375, 191, 189, 243, 261, 941, 871, 711, 812, 792, 985, 962, 677, 581, 478, 579, 457, 463, 651, 555, 568, 753, 865, 735, 833, 482, 495]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete: 21.844430446624756
[125, 156, 182, 224, 277, 362, 388, 391, 426, 454, 478, 566, 584, 634, 659, 668, 683, 728, 736, 773, 872, 887, 929, 937, 958, 963, 976, 898, 846, 869, 814, 316, 696, 286, 741, 855, 981, 492, 597, 712, 915, 757, 942, 764, 994, 232, 552, 461, 622, 327, 353, 251, 265, 248, 375, 339, 213, 193, 299, 174, 167, 344, 149, 671, 579, 485, 433, 521, 617, 535, 419, 518, 543, 447, 64

In [115]:
print(len(datapoints))

5758
